In [2]:
import sys
sys.version

'3.7.3 (default, Mar 27 2019, 22:11:17) \n[GCC 7.3.0]'

In [3]:
import torch
print(torch.__version__)

1.0.1.post2


In [4]:
#It's important that you have a working NVidia GPU set up. 
#The programming framework used to behind the scenes to work with NVidia GPUs is called CUDA. 
#Therefore, you need to ensure the following line returns True before you proceed. 
#If you have problems with this, please check the FAQ and ask for help on the forums.


In [5]:
#import tensorflow as tf
#tf.test.gpu_device_name()
torch.cuda.is_available()

False

In [6]:
from fastai.learner import *
from fastai.column_data import *

ModuleNotFoundError: No module named 'fastai.learner'

In [8]:
path='data/'
!ls data

ls: cannot access 'data': No such file or directory


In [ ]:
songs = pd.read_csv(path+'songs.csv')
songs.head()

In [ ]:
songs.shape

In [ ]:
songs.info()

In [ ]:
# If there are recurring listening event(s) triggered within a month after the user’s very first
# observable listening event, its target is marked 1, and 0 otherwise in the training set.
train = pd.read_csv(path+'train.csv')
train.head()

In [ ]:
train.shape

In [ ]:
members = pd.read_csv(path+'members.csv')
members.head()

In [ ]:
members.shape

In [ ]:
song_extra_info = pd.read_csv(path+'song_extra_info.csv')
song_extra_info.head()

In [ ]:
song_extra_info.shape

In [ ]:
test = pd.read_csv(path+'test.csv')
test.head()

In [ ]:
test.shape

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

#Here follows the distribution of the members ages. As you can see there are several with age set to 0. 
#That is obsviously something we'll have to manage.

# bd = age
f,axarray = plt.subplots(1,1,figsize=(15,10))
agehist = members.groupby(['bd'],as_index=False).count()
sns.barplot(x=agehist['bd'],y=agehist['gender'])

In [ ]:
#Members come from 21 different cities identified with an integer index ranging from 1 to 21. 
#Here is the distribution of the members cities.
f,axarray = plt.subplots(1,1,figsize=(15,10))
cityhist = members.groupby(['city'],as_index=False).count()
sns.barplot(x=cityhist['city'],y=cityhist['gender'])

In [ ]:
## Songs

print ("There are:",len(songs),"songs;",len(songs['composer'].unique()),
       "composers for",len(songs['genre_ids'].unique()),"genres in",
      len(songs['language'].unique()),"languages")

In [ ]:
#The following returns summarized aggregate information to each table accross each field.
display(DataFrameSummary(train).summary())

In [ ]:
train.info()

In [ ]:
train.describe()

In [ ]:
train['msno'].value_counts()

In [ ]:
display(DataFrameSummary(songs).summary())

In [ ]:
display(DataFrameSummary(members).summary())

# Reducing Train data set for quick evaluation
## from 7.3 mln to 100K

In [ ]:
smallTrain = train.head(n=100000)
smallTrain.to_csv(path+'small_train.csv',encoding='utf-8')
train.shape, smallTrain.shape

In [ ]:
#create a crosstab
g=smallTrain.groupby('msno')['target'].count()
topUsers=g.sort_values(ascending=False)[:15]

g=smallTrain.groupby('song_id')['target'].count()
topMovies=g.sort_values(ascending=False)[:15]

top_r = smallTrain.join(topUsers, rsuffix='_r', how='inner', on='msno')
top_r = top_r.join(topMovies, rsuffix='_r', how='inner', on='song_id')

pd.crosstab(top_r.msno, top_r.song_id, top_r.target, aggfunc=np.sum)

In [ ]:
#create the complete dataset we have to mearge the train, members and songs dataframe.

FullDF = train.merge(members,how='inner',on='msno')
FullDF = train.merge(members,how='inner',on='msno')
FullDF.head()

In [ ]:
FullDF = FullDF.merge(songs,how='inner',on='song_id')
FullDF.head()

In [ ]:
train.shape, FullDF.shape

In [7]:
FullDF.info()

NameError: name 'FullDF' is not defined

In [ ]:
display(DataFrameSummary(FullDF).summary())

# Splitting dataset train into Train and Test


In [ ]:
#FullDF['msno'].dtype
#FullDF2=FullDF
#FullDF2.head()
#FullDF = FullDF2

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

for col in FullDF.columns:
    if FullDF[col].dtype == object:
        #print(FullDF[col])
        #FullDF[col] = FullDF[col].astype('category')
        FullDF[col] = le.fit_transform(FullDF[col].astype(str))
        #FullDF[col] = le.fit_transform(FullDF[col])
    else:
        pass

In [ ]:
#FullDF2 = FullDF.fillna(FullDF.mean(), inplace=True)

In [ ]:
FullDF.info()

In [ ]:
FullDF.head()

In [ ]:
# reduce total dataset to get faster initial resutls
FullDF = FullDF.head(n=100000)

In [9]:
from sklearn.model_selection import train_test_split

y=FullDF.target
X=FullDF.drop('target',axis=1)

NameError: name 'FullDF' is not defined

In [ ]:
y.head()

In [ ]:
 X.head()

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2)
print("\nX_train:\n")
#print(X_train.head())
print(X_train.shape)

print("\nX_test:\n")
#print(X_test.head())
print(X_test.shape)


X_train:

(2, 1)

X_test:

(1, 1)


In [ ]:
X_train.fillna(X_train.mean(), inplace=True)
y_train.fillna(y_train.mean(), inplace=True)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
rnd_clf.fit(X_train, y_train)

In [ ]:
X_test.fillna(X_test.mean(), inplace=True)

In [ ]:
y_pred_rf = rnd_clf.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred_rf)

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, y_pred_rf)

In [ ]:
from sklearn.metrics import mean_squared_log_error
mean_squared_log_error(y_test, y_pred_rf)

In [ ]:
# wtf is active index?
# We're going to set the active index to Date.
# df = df.set_index("Date")